# In-Context Learning and Prompting


### Basic prompting

The simplest way to use a language model: provide a prompt `x` and sample a completion `y ~ p(y|x)`. The model treats the prompt as a prefix and
generates a continuation based on patterns learned during pretraining.

In [1]:
import transformers

from transformers import AutoTokenizer, AutoModelForCausalLM

model = "HuggingFaceTB/SmolLM2-360M"

tokenizer = AutoTokenizer.from_pretrained(model)
model = AutoModelForCausalLM.from_pretrained(model)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/831 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/689 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/724M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

#### Make a prompt `x` and tokenize it

In [2]:
x = "When a dog sees a squirrel, it will usually"

inputs = tokenizer(x, return_tensors='pt')
inputs

{'input_ids': tensor([[ 2427,   253,  2767, 10413,   253, 27721,    28,   357,   523,  2007]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

#### Generate a response

Here generating means autoregressive sampling, i.e.

```
context = `x`
for t in 0 .. max_new_tokens:
    Sample next token, y_t ~ p(y_t|context)
    Append y_t to context
```

do_sample: if True, the model samples from the probability distribution. If False, it uses greedy decoding.
temperature: scales the distribution before sampling.

*   smaller than 1.0 → sharper, more deterministic.
*   bigger than 1.0 → more random, creative.

top_k: restricts sampling to the top k most likely tokens. (top_k=50 is common).

top_p (aka nucleus sampling): restricts sampling to the smallest set of tokens whose probabilities sum to p. (top_p=0.9 is common).

In [3]:
outputs = model.generate(
    **inputs,
    max_new_tokens=20,
    do_sample=True,
    num_return_sequences=5,
    pad_token_id=tokenizer.eos_token_id
)

for i in range(5):
    print(f"===={i}====")
    print(tokenizer.decode(outputs[i]))

====0====
When a dog sees a squirrel, it will usually bark and chase it. Dogs that chase squirrels tend to do this mostly when the squirrel is on the
====1====
When a dog sees a squirrel, it will usually either crouch down quickly or turn away from the sight of the intruder.  While this
====2====
When a dog sees a squirrel, it will usually run away to avoid the danger. But some squirrels have a different reaction.

Some squirrels will
====3====
When a dog sees a squirrel, it will usually stay away from it. But it's common for them to chase squirrels and take them into their territory
====4====
When a dog sees a squirrel, it will usually try to chase the squirrel. Dog owners should keep their dogs safe around these animals by making them feel


In [6]:
outputs = model.generate(
    **inputs,
    max_new_tokens=20,
    do_sample=True,
    num_return_sequences=3,
    top_p=0.9,
    pad_token_id=tokenizer.eos_token_id
)

for i in range(3):
    print(f"===={i}====")
    print(tokenizer.decode(outputs[i]))

====0====
When a dog sees a squirrel, it will usually ignore it. This is because they are a visual predator, and a visual predator has a harder time
====1====
When a dog sees a squirrel, it will usually try to catch it. It will go into what they call ‘sit and watch’ mode. They
====2====
When a dog sees a squirrel, it will usually turn and run away. The squirrel is simply being curious and doesn't really care what other people think


In [9]:
outputs = model.generate(
    **inputs,
    max_new_tokens=20,
    do_sample=True,
    num_return_sequences=3,
    temperature=1.9,
    pad_token_id=tokenizer.eos_token_id
)

for i in range(3):
    print(f"===={i}====")
    print(tokenizer.decode(outputs[i]))

====0====
When a dog sees a squirrel, it will usually try and make physical contact as close to that squirrel as to appear threatening. Some owners mistakenly train their
====1====
When a dog sees a squirrel, it will usually freeze and assume something very strange; i. e that she may see some part at an end being
====2====
When a dog sees a squirrel, it will usually run down to ground to catch and steal its meal on spot and take of fast from it in most


### Instruction prompt ("zero shot")

Instead of just continuing text, we can prompt the model to perform a specific task by providing an instruction. This is "zero-shot" because we give no examples, just the task description. The model uses any instruction-following related patterns it learned during training.

In [10]:
prompt_template = """Classify the sentence's sentiment as 'Positive' or 'Negative':
{sentence}
Classification:"""


sentences = [
    "I love the NLP class!",
    "I didn't race well and lost :("
]

prompt = prompt_template.format(sentence=sentences[0])
print(prompt)


Classify the sentence's sentiment as 'Positive' or 'Negative':
I love the NLP class!
Classification:


In [15]:
for sentence in sentences:
    print(f"\n=============")

    prompt = prompt_template.format(sentence=sentence)
    inputs = tokenizer(prompt, return_tensors='pt')
    outputs = model.generate(
        **inputs,
        max_new_tokens=40,
        do_sample=True,
        num_return_sequences=2,
        pad_token_id=tokenizer.eos_token_id
    )

    for i in range(2):
        print(f"----{i}----")
        print(tokenizer.decode(outputs[i]))


----0----
Classify the sentence's sentiment as 'Positive' or 'Negative':
I love the NLP class!
Classification: Positive I love the NLP class.

In the previous exercise, we created the sentiment class:

Now we will create a sentiment model for our movie review classification problem. Recall that the sentiment
----1----
Classify the sentence's sentiment as 'Positive' or 'Negative':
I love the NLP class!
Classification:   Positive

I love the NLP class!<|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|>

----0----
Classify the sentence's sentiment as 'Positive' or 'Negative':
I didn't race well and lost :(
Classification: Positive
Explanation: I didn't race well and l

It's important to ensure that the output is formatted correctly!

### Instruction + examples ("few-shot")

We can provide examples of input-output pairs before the test input. This "few-shot" or "in-context learning" approach helps the model understand the task format and expected outputs without any parameter updates.

In [16]:
prompt_template = """Classify the sentence's sentiment as 'Positive' or 'Negative'. Examples:

Sentence:
This is such a cool lecture!
Classification:
Positive

Sentence:
I really don't like the last scene.
Classification:
Negative

Sentence:
{sentence}
Classification:
"""



In [17]:
for sentence in sentences:
    print(f"\n=============")
    prompt = prompt_template.format(sentence=sentence)
    inputs = tokenizer(prompt, return_tensors='pt')
    outputs = model.generate(
        **inputs,
        max_new_tokens=20,
        do_sample=True,
        num_return_sequences=2,
        stop_strings=["\n\n"],
        tokenizer=tokenizer,
        pad_token_id=tokenizer.eos_token_id
    )

    for i in range(2):
        print(f"----{i}----")
        print(tokenizer.decode(outputs[i]))


----0----
Classify the sentence's sentiment as 'Positive' or 'Negative'. Examples:

Sentence:
This is such a cool lecture!
Classification:
Positive

Sentence:
I really don't like the last scene.
Classification:
Negative

Sentence:
I love the NLP class!
Classification:
Positive


----1----
Classify the sentence's sentiment as 'Positive' or 'Negative'. Examples:

Sentence:
This is such a cool lecture!
Classification:
Positive

Sentence:
I really don't like the last scene.
Classification:
Negative

Sentence:
I love the NLP class!
Classification:
Positive



----0----
Classify the sentence's sentiment as 'Positive' or 'Negative'. Examples:

Sentence:
This is such a cool lecture!
Classification:
Positive

Sentence:
I really don't like the last scene.
Classification:
Negative

Sentence:
I didn't race well and lost :(
Classification:
Negative

<|endoftext|>
----1----
Classify the sentence's sentiment as 'Positive' or 'Negative'. Examples:

Sentence:
This is such a cool lecture!
Classificatio

### Chat templates

Some models have been fine-tuned to operate as chat assistants. The chat is represented as a series of messages that are turned into a string using special tags. There is also a *system message* that provides instructions about how the model should behave.

These models are often called "instruct" models because they've been trained to follow instructions rather than just complete text.

In [18]:
import transformers

from transformers import AutoTokenizer, AutoModelForCausalLM

model = "HuggingFaceTB/SmolLM2-360M-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model)
model = AutoModelForCausalLM.from_pretrained(model)

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/655 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/846 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/724M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

In [20]:
messages = [{
    "role": "user",
    "content": "What is the capital of France."
}]

input_text = tokenizer.apply_chat_template(messages, tokenize=False)
print("Input text: ", input_text, sep="\n")


Input text: 
<|im_start|>system
You are a helpful AI assistant named SmolLM, trained by Hugging Face<|im_end|>
<|im_start|>user
What is the capital of France.<|im_end|>



Generate a response

In [21]:
inputs = tokenizer(input_text, return_tensors="pt")
outputs = model.generate(**inputs, max_new_tokens=50, do_sample=True)
print(tokenizer.decode(outputs[0]))

<|im_start|>system
You are a helpful AI assistant named SmolLM, trained by Hugging Face<|im_end|>
<|im_start|>user
What is the capital of France.<|im_end|>
<|im_start|>assistant
Based on my training data, the capital of France is Paris.<|im_end|>


### System prompts

Chat models often support a system message that sets the model's behavior or role. This message is typically prepended to the conversation and instructs the model how to respond throughout the interaction. For example, we can use the system prompt to have the model respond in French.

In [22]:
messages = [
    {
        "role": "system",
        "content": "You are an assistant that speaks in French."
    },
    {
        "role": "user",
        "content": "What is the capital of France."
    }
]

input_text = tokenizer.apply_chat_template(messages, tokenize=False)
inputs = tokenizer(input_text, return_tensors="pt")
outputs = model.generate(**inputs, max_new_tokens=50, do_sample=True)
print(tokenizer.decode(outputs[0]))

<|im_start|>system
You are an assistant that speaks in French.<|im_end|>
<|im_start|>user
What is the capital of France.<|im_end|>
<|im_start|>assistant
La Ville de Paris, or the Capital City, est de l'Océanie, que le Consulat s'occupera de cette cité.<|im_end|>


### Instruction ("zero shot")

Using the chat format for zero-shot tasks. The instruction-tuned model may follow instructions more reliably than the base model, though output formatting can still be inconsistent.

In [24]:
messages = [
    {
        "role": "user",
        "content": ("Classify the sentence's sentiment as 'Positive' or 'Negative':\n" +
                    "Sentence: 'I love the NLP class!'\n")
    }
]

input_text = tokenizer.apply_chat_template(messages, tokenize=False)
inputs = tokenizer(input_text, return_tensors="pt")
outputs = model.generate(**inputs, max_new_tokens=50, do_sample=True, num_return_sequences=3)
for i in range(len(outputs)):
    print(f"----{i}----")
    print(tokenizer.decode(outputs[i]))

----0----
<|im_start|>system
You are a helpful AI assistant named SmolLM, trained by Hugging Face<|im_end|>
<|im_start|>user
Classify the sentence's sentiment as 'Positive' or 'Negative':
Sentence: 'I love the NLP class!'
<|im_end|>
<|im_start|>assistant
Positive<|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|>
----1----
<|im_start|>system
You are a helpful AI assistant named SmolLM, trained by Hugging Face<|im_end|>
<|im_start|>user
Classify the sentence's sentiment as 'Positive' or 'Negative':
Sentence: 'I love the NLP class!'
<|im_end|>
<|im_start|>assistant
Sentiment: positive.<|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><

#### Approach 1: write a detailed instruction (in the user or system prompt)

We can improve output formatting by providing explicit, detailed instructions about the desired format in either the system or user message.

In [25]:
messages = [
    {   "role": "system",
        "content": """You are an expert sentiment classifier.
Your task is to classify a sentence's sentiment as 'Positive' or 'Negative'.
The user will provide you with a sentence.
Format your output as:

Classification: Positive or Negative
"""
    },
    {
        "role": "user",
        "content": ("I love the NLP class!")
    }
]

input_text = tokenizer.apply_chat_template(messages, tokenize=False)
inputs = tokenizer(input_text, return_tensors="pt")
outputs = model.generate(**inputs, max_new_tokens=50, do_sample=True, num_return_sequences=3)
for i in range(len(outputs)):
    print(f"----{i}----")
    print(tokenizer.decode(outputs[i]))

----0----
<|im_start|>system
You are an expert sentiment classifier.
Your task is to classify a sentence's sentiment as 'Positive' or 'Negative'.
The user will provide you with a sentence.
Format your output as:

Classification: Positive or Negative
<|im_end|>
<|im_start|>user
I love the NLP class!<|im_end|>
<|im_start|>assistant
Classification: Positive<|im_end|>
----1----
<|im_start|>system
You are an expert sentiment classifier.
Your task is to classify a sentence's sentiment as 'Positive' or 'Negative'.
The user will provide you with a sentence.
Format your output as:

Classification: Positive or Negative
<|im_end|>
<|im_start|>user
I love the NLP class!<|im_end|>
<|im_start|>assistant
Classification: Positive<|im_end|>
----2----
<|im_start|>system
You are an expert sentiment classifier.
Your task is to classify a sentence's sentiment as 'Positive' or 'Negative'.
The user will provide you with a sentence.
Format your output as:

Classification: Positive or Negative
<|im_end|>
<|im_

#### Approach 2: provide examples (either in the system prompt or as a sequence of messages)

We can show the model the desired behavior through example conversations, where the assistant demonstrates the correct format and task execution. This is analogous to the few-shot examples we saw earlier, but using the chat format.

In [26]:
messages = [
    {
        "role": "user",
        "content": "Classify the sentence's sentiment as 'Positive' or 'Negative':\nSentence: 'This is such a cool lecture!'"
    },
    {
        "role": "assistant",
        "content": "Classification: Positive"
    },
    {
        "role": "user",
        "content": "Classify the sentence's sentiment as 'Positive' or 'Negative':\nSentence: 'I really don't like the last scene.'"
    },
    {
        "role": "assistant",
        "content": "Classification: Negative"
    },
    {
        "role": "user",
        "content": "Classify the sentence's sentiment as 'Positive' or 'Negative':\nSentence: 'I love the NLP class!'"
    }
]

input_text = tokenizer.apply_chat_template(messages, tokenize=False)
inputs = tokenizer(input_text, return_tensors="pt")
outputs = model.generate(**inputs, max_new_tokens=50, do_sample=True, num_return_sequences=3)
for i in range(len(outputs)):
    print(f"----{i}----")
    print(tokenizer.decode(outputs[i]))

----0----
<|im_start|>system
You are a helpful AI assistant named SmolLM, trained by Hugging Face<|im_end|>
<|im_start|>user
Classify the sentence's sentiment as 'Positive' or 'Negative':
Sentence: 'This is such a cool lecture!'<|im_end|>
<|im_start|>assistant
Classification: Positive<|im_end|>
<|im_start|>user
Classify the sentence's sentiment as 'Positive' or 'Negative':
Sentence: 'I really don't like the last scene.'<|im_end|>
<|im_start|>assistant
Classification: Negative<|im_end|>
<|im_start|>user
Classify the sentence's sentiment as 'Positive' or 'Negative':
Sentence: 'I love the NLP class!'<|im_end|>
<|im_start|>assistant
Classification: Positive<|im_end|>
----1----
<|im_start|>system
You are a helpful AI assistant named SmolLM, trained by Hugging Face<|im_end|>
<|im_start|>user
Classify the sentence's sentiment as 'Positive' or 'Negative':
Sentence: 'This is such a cool lecture!'<|im_end|>
<|im_start|>assistant
Classification: Positive<|im_end|>
<|im_start|>user
Classify the se

### Chain-of-thought with base model

Prompting the model to "think step by step" can elicit intermediate reasoning steps before the final answer. Even base models can exhibit this behavior when prompted appropriately, though the reasoning may be flawed.

In [27]:
model = "HuggingFaceTB/SmolLM2-360M"

tokenizer = AutoTokenizer.from_pretrained(model)
model = AutoModelForCausalLM.from_pretrained(model)

In [28]:
prompts = [
    """Q: On average Joe throws 25 punches per minute.
A fight lasts 5 rounds of 3 minutes.
How many punches did he throw?
A: Let's think step by step.""",
]

for prompt in prompts:
    inputs = tokenizer(prompt, return_tensors="pt")
    outputs = model.generate(
        **inputs,
        pad_token_id=tokenizer.eos_token_id,
        max_new_tokens=512,
        temperature=0.4,
        do_sample=True
    )
    print(tokenizer.decode(outputs[0]))
    print("====")

Q: On average Joe throws 25 punches per minute. 
A fight lasts 5 rounds of 3 minutes. 
How many punches did he throw?
A: Let's think step by step.
1. The first round lasts 3 minutes, so 3 minutes * 25 punches per minute = 75 punches.
2. The second round lasts 3 minutes, so 3 minutes * 25 punches per minute = 75 punches.
3. The third round lasts 3 minutes, so 3 minutes * 25 punches per minute = 75 punches.
4. The fourth round lasts 3 minutes, so 3 minutes * 25 punches per minute = 75 punches.
5. The fifth round lasts 3 minutes, so 3 minutes * 25 punches per minute = 75 punches.
6. The fight lasts 5 rounds, so 5 * 3 minutes * 25 punches per minute = 75 * 5 punches.
7. The total number of punches Joe threw in the fight is 75 * 5 punches.

Q: What is the probability that Joe will lose the fight?
A: Let's think step by step.
1. The first round lasts 3 minutes, so 3 minutes * 25 punches per minute = 75 punches.
2. The second round lasts 3 minutes, so 3 minutes * 25 punches per minute = 75 pu

### Chain-of-thought with instruct model

Many instruction-tuned models can solve problems step-by-step when asked, as they've typically been trained to follow such instructions.

In [29]:
model = "HuggingFaceTB/SmolLM2-360M-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model)
model = AutoModelForCausalLM.from_pretrained(model)

In [30]:
messages = [
    {
        "role": "user",
        "content": """Solve the problem:
On average Joe throws 25 punches per minute.
A fight lasts 5 rounds of 3 minutes.
How many punches did he throw?"""
    }
]

input_text = tokenizer.apply_chat_template(messages, tokenize=False)
inputs = tokenizer(input_text, return_tensors="pt")
outputs = model.generate(**inputs, max_new_tokens=256, temperature=0.4, do_sample=True)
print(tokenizer.decode(outputs[0]))

<|im_start|>system
You are a helpful AI assistant named SmolLM, trained by Hugging Face<|im_end|>
<|im_start|>user
Solve the problem:
On average Joe throws 25 punches per minute. 
A fight lasts 5 rounds of 3 minutes. 
How many punches did he throw?<|im_end|>
<|im_start|>assistant
To find out how many punches Joe threw, we need to calculate the total number of punches he threw in the fight.

First, we need to find out how many rounds there are in the fight. Since Joe throws 25 punches per minute, and there are 60 minutes in an hour, there are 60 * 5 = 300 rounds in the fight.

Now, we need to find out how many punches there are in each round. Since there are 300 rounds, and Joe throws 25 punches per minute, there are 300 / 25 = 12 punch per round.

So, Joe threw 12 * 3 = 36 punches in the fight.<|im_end|>


### Program-aided reasoning

Instead of natural language reasoning, we can prompt the model to solve problems by writing and executing code. This leverages the model's code generation capabilities and the code executor's accurate computations.

In [31]:
messages = [
    {
        "role": "user",
        "content": """Solve the problem by writing a Python program:
On average Joe throws 25 punches per minute.
A fight lasts 5 rounds of 3 minutes.
How many punches did he throw?"""
    }
]

input_text = tokenizer.apply_chat_template(messages, tokenize=False)
inputs = tokenizer(input_text, return_tensors="pt")
outputs = model.generate(**inputs, max_new_tokens=512, temperature=0.4, do_sample=True)
print(tokenizer.decode(outputs[0]))

<|im_start|>system
You are a helpful AI assistant named SmolLM, trained by Hugging Face<|im_end|>
<|im_start|>user
Solve the problem by writing a Python program:
On average Joe throws 25 punches per minute. 
A fight lasts 5 rounds of 3 minutes. 
How many punches did he throw?<|im_end|>
<|im_start|>assistant
Here's a Python program that solves this problem:

```python
import time

def punch_thrower(minutes):
    total_punches = 25 * minutes
    return total_punches

def main():
    minutes = 5
    punch_thrower_count = punch_thrower(minutes)
    print(f"On average, Joe throws {punch_thrower_count} punches per minute.")
    print(f"A fight lasts {minutes * 3} minutes. Joe threw {punch_thrower_count} punches.")

if __name__ == "__main__":
    main()
```

This program defines a function `punch_thrower` that takes the number of minutes as an argument and returns the number of punches Joe throws. The `main` function sets up the program and calls the `punch_thrower` function with the given nu